In [13]:
from selenium.webdriver import Chrome
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from fake_useragent import UserAgent
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from datetime import datetime, date, timedelta
from itertools import chain
from collections import Counter
import time, os, psycopg2, json, requests, re
import pandas as pd

from bs4 import BeautifulSoup
import numpy as np

### Historic Match Scores & Details Extraction (Website: https://www.flashscore.nl/)

In [14]:
def matches_scores(url):
    '''This function extracts the the historic match scores of the teams paired up for an upcoming match
    and transforms the data for further analaysis.'''
    options = Options()
    
    #Sets up a fake browser
    ua = UserAgent()
    userAgent = ua.random
    options.add_argument(f'user-agent={userAgent}')
    options.add_argument('--blink-settings=imagesEnabled=false')
    
    #capa = DesiredCapabilities.CHROME
    #capa["pageLoadStrategy"] = "none"
    #,desired_capabilities=capa
    driver = webdriver.Chrome(r'C:\Users\HP\Downloads\Premier_League_Stats\Loading_with_selenium\chromedriver.exe',options=options)
    driver.get(url)
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="onetrust-accept-btn-handler"]'))).click()
    #WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CLASS_NAME, "h2h")))
    #driver.execute_script("window.stop();")
    
    #sleeps because the page has to load the data before scraping h2h__showMore showMore
    
    elements = driver.find_elements(By.CLASS_NAME, "showMore")
    for elem in elements:
        elem.click()
    contents =driver.find_elements_by_class_name("h2h")
    
    #gets the data by class name
    content = [con.get_attribute('innerText') for con in contents]
    
    #extracts the content of each H2H page and cleans it
    clean = [item.replace('\n', ',') for item in content]
    clean = [item.split(',') for item in clean]
    clean = list(chain(*clean))
    remove = ['Toon meer wedstrijden']
    clean  = [word for word in clean if word not in remove ]
    
    driver.quit()
    
    #Using the table sub-table heads to split the data between the last 10 matches for the home and away teams and their head2head
    start_indices = [i for i, item in enumerate(clean) if "LAATSTE WEDSTRIJDEN" in item or "HEAD-TO-HEAD" in item]
    end_indices = start_indices[1:] + [len(clean)]
    first_list = clean[start_indices[0]:end_indices[0]]
    second_list = clean[start_indices[1]:end_indices[1]]
    third_list = clean[start_indices[2]:end_indices[2]]

    #Removes the sub-table heads to only have the useful information for the analysis
    first_list = [word for word in first_list if 'LAATSTE WEDSTRIJDEN' not in word]
    second_list = [word for word in second_list if 'LAATSTE WEDSTRIJDEN' not in word]
    third_list = [word for word in third_list if 'HEAD-TO-HEAD' not in word]

    #Adds all the home historic match outcomes to a dictionary
    home_team_matches = {'date': [],'league': [],'home_club': [],'away_club': [],'home_club_goal': [],'away_club_goal': []}
    keys = list(home_team_matches.keys())
    count_1 = 0
    for item in first_list:
        if count_1 <= 5:
            home_team_matches[keys[count_1]].append(item)
            count_1 += 1
        else:
            #Account for instance there are some unexpected values within the data being extracted
            try:
                value = int(item)
                continue
            except:
                count_1 = 0
                continue
    home_team_matches = json.dumps(home_team_matches)

    #Adds all the away historic match outcomes to a dictionary
    away_team_matches = {'date': [],'league': [],'home_club': [],'away_club': [],'home_club_goal': [],'away_club_goal': []}
    keys = list(away_team_matches.keys())
    count_1 = 0
    for item in second_list:
        if count_1 <= 5:
            away_team_matches[keys[count_1]].append(item)
            count_1 += 1
        else:
            #Account for instance there are some unexpected values within the data being extracted
            try:
                value = int(item)
                continue
            except:
                count_1 = 0
                continue
    away_team_matches = json.dumps(away_team_matches)

    #Adds all the head2head historic match outcomes to a dictionary
    head2head_matches = {'date': [],'league': [],'home_club': [],'away_club': [],'home_club_goal': [],'away_club_goal': []}
    keys = list(head2head_matches.keys())
    count_1 = 0
    for item in third_list:
        if count_1 <= 5:
            head2head_matches[keys[count_1]].append(item)
            count_1 += 1
        else:
            #Account for instance there are some unexpected values within the data being extracted
            if len(item) > 3:
                head2head_matches['date'].append(item)
                count_1 = 1
            else:    
                continue
    head2head_matches = json.dumps(head2head_matches)

    #Account for instances where no matches have been played
    return home_team_matches, away_team_matches, head2head_matches   

In [15]:
def activity_times(content):
    '''Takes in the match content and extracts the time (also calculated the overtime)'''
    activity_time = [con.replace("\n","'") for con in content]
    activity_time = [con.split("'")[0] for con in activity_time]
    for ind in range(len(activity_time)):
        if '+' in activity_time[ind]:
            temp_vars = activity_time[ind].split('+')
            temp_var = int(temp_vars[0])+int(temp_vars[1])
            activity_time[ind] = str(temp_var)
    return(activity_time)

In [16]:
def activity_type(activities):
    '''Takes in the content of the match, cleans it and extract the type of activity'''
    activities_list = []
    delim = r'[<>]'
    diction = {'card':'red-yellowcard','red':'redcard','soccer':'goal','substitution':'substitution','var':'var','warning':'penalty(missed)','yellow':'yellowcard'}
    for activity in activities:
        temp_list = [i for i in re.split(delim, activity) if 'svg class' in i]
        
        chosen_activity = []
        for key in diction.keys():
            if key in temp_list[0]:
                #This specifically checks for red-yellow card, as it is difficult to distinguish from red or yellow card by just 'svg class'
                if key == 'card':
                    if ('red' not in temp_list[0]) & ('yellow' not in temp_list[0]):
                        chosen_activity.append(diction['card'])
                else:
                    chosen_activity.append(diction[key])
                    
        activities_list.append(chosen_activity[0])
    return(activities_list)

In [17]:
def matches_details(team, url):
    '''This function simply extracts the inner match details of the historic matches of the teams under analysis'''
    team = team
    url = url
    
    #Sets up a fake browser
    options = Options()
    ua = UserAgent()
    userAgent = ua.random
    options.add_argument(f'user-agent={userAgent}')
    #options.add_argument('--blink-settings=imagesEnabled=false')
    #options.add_argument("--headless")

    #capa = DesiredCapabilities.CHROME
    #capa["pageLoadStrategy"] = "none"
    #desired_capabilities=capa
    driver = webdriver.Chrome(r'C:\Users\HP\Downloads\Premier_League_Stats\Loading_with_selenium\chromedriver.exe',options=options)
    #Loads up the url using the chromedriver and clicks the cookie prompt
    driver.get(league_url) 
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="onetrust-accept-btn-handler"]'))).click()
    WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CLASS_NAME, "smv__participantRow.smv__awayParticipant")))
    #driver.execute_script("window.stop();")
    #Extracts the data in the table rows (containing links to previous match)
    elements = driver.find_elements(By.CLASS_NAME, "h2h__row")#'Klik voor wedstrijddetails!'
    
    list_of_details = {}
    count = 0
    
    #Loops through each match link to extract the contents within each page (match details)
    for elem in elements:
        count += 1
        
        driver.switch_to.window(driver.window_handles[0]) #Switches the driver to base page (the page it started with)
        elem.click() #Click the match link which opens up a new page
        driver.switch_to.window(driver.window_handles[-1]) #Switches the driver to the new opened page
        time.sleep(4) #waits for the page to load
        
        #checks which role the team under consideration played as (home or away) in the match being checked
        try:
            match_date = driver.find_element_by_class_name('duelParticipant__startTime')
            match_date = match_date.get_attribute('textContent')

            tournament = driver.find_element_by_class_name('tournamentHeader__country')
            tournament = tournament.get_attribute('textContent')
            
            match_score = driver.find_element_by_class_name('detailScore__wrapper')
            match_score = match_score.get_attribute('textContent')

            try:
                home_team = driver.find_element_by_class_name('duelParticipant__home')
                home_teamname = home_team.get_attribute('textContent')
            except:
                home_team = driver.find_element_by_class_name('duelParticipant__home duelParticipant--winner')
                home_teamname = home_team.get_attribute('textContent')
            try:
                away_team = driver.find_element_by_class_name('duelParticipant__away ')
                away_teamname = away_team.get_attribute('textContent')
            except:
                away_team = driver.find_element_by_class_name('duelParticipant__away duelParticipant--winner')
                away_teamname = away_team.get_attribute('textContent')

            #Based on which role, the class name is decided and used to extract details
            if team in home_teamname:
                class_name_1 = 'smv__participantRow.smv__homeParticipant'
                class_name_2 = 'smv__participantRow.smv__awayParticipant'
            elif team in away_teamname:
                class_name_1 = 'smv__participantRow.smv__awayParticipant'
                class_name_2 = 'smv__participantRow.smv__homeParticipant'
            
            #Extracts the contents fo the match
            contents_1 = driver.find_elements_by_class_name(class_name_1)
            content_1 = [con.get_attribute('innerText') for con in contents_1]
            activities_1 = [con.get_attribute('innerHTML') for con in contents_1]

            #Extracts the activities time and type and zips the lists to be processed together
            activities_time_1 = activity_times(content_1)
            activities_list_1 = activity_type(activities_1)
            combined_list_1 = list(zip(activities_time_1, activities_list_1))

            #Groups the match activities time by the type of activity (yellow card, substitution etc.)
            details_1 = {'tournament':[tournament],'date':[match_date],'teams':[home_teamname,away_teamname],
                       'match_score':[match_score],'goal':[],'penalty(missed)':[],'redcard':[],'red-yellowcard':[],'substitution':[],
                       'var':[],'yellowcard':[]}
            for entry in combined_list_1:
                details_1[entry[1]].append(entry[0])
                
            #Extracts the contents fo the match
            contents_2 = driver.find_elements_by_class_name(class_name_2)
            content_2 = [con.get_attribute('innerText') for con in contents_2]
            activities_2 = [con.get_attribute('innerHTML') for con in contents_2]

            #Extracts the activities time and type and zips the lists to be processed together
            activities_time_2 = activity_times(content_2)
            activities_list_2 = activity_type(activities_2)
            combined_list_2 = list(zip(activities_time_2, activities_list_2))

            #Groups the match activities time by the type of activity (yellow card, substitution etc.)
            details_2 = {'tournament':[tournament],'date':[match_date],'teams':[home_teamname,away_teamname],
                       'match_score':[match_score],'goal':[],'penalty(missed)':[],'redcard':[],'red-yellowcard':[],'substitution':[],
                       'var':[],'yellowcard':[]}
            for entry in combined_list_2:
                details_2[entry[1]].append(entry[0])

            #Appends to the list of details which will be changed to dictionary
            list_of_details[str(count)] = {}
            list_of_details[str(count)]['team'] = details_1
            list_of_details[str(count)]['opponent'] = details_2
            driver.close()
        except:
            list_of_details[str(count)] = {}
            driver.close()      
    driver.quit()
    list_of_details = json.dumps(list_of_details)

    return list_of_details 

In [18]:
def match_extraction(leagues_list, today_date):
    leagues_dataset = {} #Created the empyt dictionary that will be used to concatenate all table from all leagues
    
    for key in list(leagues_list.keys()):
        try:
            '''To make sure all links load, irrespective of poor network or site loading wrongly,
            add a while loop which checks if a variable has been changed. If site loads properly,
            change variable to exit while loop, but if webiste threw an except (Timeout) message,
            keep variable the same to maintain the while loop until data is gotten'''
            
            league_url = leagues_list[key][0]

            #Sets up a fake browser
            options = Options()
            ua = UserAgent()
            userAgent = ua.random
            options.add_argument(f'user-agent={userAgent}')
            options.add_argument('--blink-settings=imagesEnabled=false')
            #options.add_argument("--headless")

            #capa = DesiredCapabilities.CHROME
            #capa["pageLoadStrategy"] = "none"
            #desired_capabilities=capa

            #driver = webdriver.Chrome(r'C:\Users\hp\Documents\Our Documents\Personal Development\Projects\Client Projects\Dexter Hadeveld (Upwork)\Enhancing-Sports-Insight\chromedriver-win64\chromedriver.exe',options=options)
            driver = webdriver.Chrome(r'C:\Users\HP\Downloads\Premier_League_Stats\Loading_with_selenium\chromedriver.exe',options=options)
            #Loads up the url using the chromedriver and clicks the cookie prompt
            driver.get(league_url) 
            WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="onetrust-accept-btn-handler"]'))).click()
            #WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, '//*[@title ="Klik voor wedstrijddetails!"]')))
            #driver.execute_script("window.stop();")
            #Extracts the contents of the table of scheduled matches
            contents = driver.find_elements_by_xpath('//*[@title ="Klik voor wedstrijddetails!"]')
            content = [con.get_attribute('innerText') for con in contents]
            id_match = [con.get_attribute('id') for con in contents]

            driver.quit() #Closes driver to begin the transformation

            #Transformation: sets up urls to the head2head, home and away team form for previous matches
            clean= [entry.replace('\n-\n-', '').replace('\n', ',').split(',') for entry in content]
            
            #This filters the scheduled matches of any cancelled matches
            zipped_lists = zip(clean, id_match)
            zipped_lists = [pair for pair in zipped_lists if len(pair[0]) == 3]
            clean, id_match = zip(*zipped_lists)
            
            #Prepares the links needed to extract the historic match scores and in-match details
            base_url = 'https://www.flashscore.nl/wedstrijd/'
            overall_urls = [f"{base_url}{string.replace('g_1_', '')}/#/h2h/overall" for string in id_match]
            home_urls = [f"{base_url}{string.replace('g_1_', '')}/#/h2h/home" for string in id_match]
            away_urls = [f"{base_url}{string.replace('g_1_', '')}/#/h2h/away" for string in id_match]
            df = pd.DataFrame(clean, columns=['Date/Time', 'HomeTeam', 'AwayTeam'])
            df['match_urls'] = overall_urls
            df['home_urls'] = home_urls
            df['away_urls'] = away_urls
            
            #Converting the date columns to datetime
            df['Date/Time'] = pd.to_datetime(df['Date/Time'] + '.2023', format='%d.%m. %H:%M.%Y')

            #filters the dataframe prepared using the current date
            today_df = df[df['Date/Time'].dt.date == today_date]
            today_df = today_df.copy(deep=True)
            curr_league = [key for i in range(len(today_df['match_urls']))]
            today_df['League'] = curr_league

            hometeam_form = []
            awayteam_form = []
            head2head = []
            home_details = []
            away_detials = []

            #for each match url extract the head2head, home team and away team games score for the last 10 recent games
            for url in today_df.match_urls:
                try:
                    home_team_matches, away_team_matches, head2head_matches = matches_scores(url)
                    hometeam_form.append(home_team_matches)
                    awayteam_form.append(away_team_matches)
                    head2head.append(head2head_matches)
                except:
                    empty_json = json.dumps({})
                    hometeam_form.append(empty_json)
                    awayteam_form.append(empty_json)
                    head2head.append(empty_json)
            
            #extract the match detail (yellow cars, goals, penalties and times etc) for last 10 games by home team
            for i in range(len(list(today_df['home_urls']))):
                home_url = list(today_df['home_urls'])[i]
                home_team = list(today_df['HomeTeam'])[i]
                try:
                    home_team_dets = matches_details(home_team, home_url)
                    home_details.append(home_team_dets)
                except:
                    empty_json = json.dumps({})
                    home_details.append(empty_json)
            
            #extract the match detail (yellow cars, goals, penalties and times etc) for last 10 games by away team
            for i in range(len(list(today_df['away_urls']))):
                away_url = list(today_df['away_urls'])[i]
                away_team = list(today_df['AwayTeam'])[i]
                try:
                    away_team_dets = matches_details(away_team, away_url)
                    away_detials.append(away_team_dets)
                except:
                    empty_json = json.dumps({})
                    away_detials.append(empty_json)

            #Add all these extracted data to the dataframe of daily match of the current league being extracted
            today_df['home_team_matches'] = hometeam_form
            today_df['away_team_matches'] = awayteam_form
            today_df['head2head_matches'] = head2head
            today_df['home_team_matchespattern'] = home_details
            today_df['away_team_matchespattern'] = away_detials
            
            
            #Add the dataframe of daily match of the league to the dictionary of dataframes
            leagues_dataset[key] = today_df
            print("All daily matches of {} have been extracted!".format(key))
        except:
            try:
                driver.quit()
                print("Daily matches of {} couldn't be extracted!".format(key))
            except:
                print("Daily matches of {} couldn't be extracted!".format(key))
            continue
     
    #All the dataframe of the daily matches for all the leagues extracted are concatenated vertically
    list_of_keys = list(leagues_dataset.keys())
    if len(list_of_keys) > 0:
        for i in range(len(list_of_keys)):
            if i == 0:
                key = list_of_keys[i]
                final_dataset = leagues_dataset[key].copy(deep=True)
            else:
                key = list_of_keys[i]
                final_dataset = pd.concat([final_dataset, leagues_dataset[key]], axis=0)

        return final_dataset
    else:
        print('No league scheduled matches were extracted!')
        

In [19]:
#Dictionary containing the urls of each league for the different sites extracted from.

leagues_list = {
    'English Premier League':['https://www.flashscore.nl/voetbal/engeland/premier-league/schema/', 'https://www.worldfootball.net/all_matches/eng-premier-league-2023-2024/'],
    'Championship':['https://www.flashscore.nl/voetbal/engeland/championship/schema/','https://www.worldfootball.net/all_matches/eng-championship-2023-2024/'],
    'EFL Championship':['https://www.flashscore.nl/voetbal/engeland/efl-cup/schema/', ''],
    'EFL Trophy':['https://www.flashscore.nl/voetbal/engeland/efl-trophy/schema/', 'https://www.worldfootball.net/all_matches/eng-efl-trophy-2023-2024/'],
    'La Liga':['https://www.flashscore.nl/voetbal/spanje/laliga/schema/','https://www.worldfootball.net/all_matches/esp-primera-division-2023-2024/'],
    'La Liga2':['https://www.flashscore.nl/voetbal/spanje/laliga2/schema/', 'https://www.worldfootball.net/all_matches/esp-segunda-division-2023-2024/'],
    'Bundesliga':['https://www.flashscore.nl/voetbal/duitsland/bundesliga/schema/', 'https://www.worldfootball.net/all_matches/bundesliga-2023-2024/'],
    'Serie A':['https://www.flashscore.nl/voetbal/italie/serie-a/schema/', 'https://www.worldfootball.net/all_matches/ita-serie-a-2023-2024/'],
    'Serie B':['https://www.flashscore.nl/voetbal/italie/serie-b/schema/', 'https://www.worldfootball.net/all_matches/ita-serie-b-2023-2024/'],
    'Ligue 1':['https://www.flashscore.nl/voetbal/frankrijk/ligue-1/schema/', 'https://www.worldfootball.net/all_matches/fra-ligue-1-2023-2024/'],
    'Ligue 2':['https://www.flashscore.nl/voetbal/frankrijk/ligue-2/schema/', 'https://www.worldfootball.net/all_matches/fra-ligue-2-2023-2024/'],
    'Brasileirão Serie A':['https://www.flashscore.nl/voetbal/brazilie/braziliaanse-competitie/schema/', 'https://www.worldfootball.net/all_matches/bra-serie-a-2023/'],
    'Primera División':['https://www.flashscore.nl/voetbal/argentinie/primera-d/schema/', ''],
    'Major League Soccer':['https://www.flashscore.nl/voetbal/usa/mls/schema/', 'https://www.worldfootball.net/all_matches/usa-major-league-soccer-2023/'],
    'Eredivisie':['https://www.flashscore.nl/eredivisie/schema/', 'https://www.worldfootball.net/all_matches/ned-eredivisie-2023-2024/'],
    'Primeira Liga':['https://www.flashscore.nl/voetbal/portugal/liga-portugal/schema/', 'https://www.worldfootball.net/all_matches/por-primeira-liga-2023-2024/'],
    'J1 League':['https://www.flashscore.nl/voetbal/japan/j1-league/schema/', 'https://www.worldfootball.net/all_matches/jpn-j1-league-2023/'],
    'Scottish Premiership':['https://www.flashscore.nl/voetbal/schotland/premiership/schema/', 'https://www.worldfootball.net/all_matches/sco-premiership-2023-2024/'],
    'Superliga':['https://www.flashscore.nl/voetbal/denemarken/superliga/schema/', 'https://www.worldfootball.net/all_matches/den-superligaen-2023-2024/'],
    'Süper Lig':['https://www.flashscore.nl/voetbal/turkije/super-lig/schema/', 'https://www.worldfootball.net/all_matches/tur-sueperlig-2023-2024/'],
    'Allsvenskan':['https://www.flashscore.nl/voetbal/zweden/allsvenskan/schema/', 'https://www.worldfootball.net/all_matches/swe-allsvenskan-2023/'],
    'Saudi Professional League':['https://www.flashscore.nl/voetbal/saoedi-arabie/premier-league/schema/', 'https://www.worldfootball.net/all_matches/ksa-saudi-pro-league-2023-2024/'],
    'Jupiler Pro League':['https://www.flashscore.nl/voetbal/belgie/jupiler-pro-league/schema/', 'https://www.worldfootball.net/all_matches/bel-eerste-klasse-a-2023-2024/'],
    'UEFA Champions League':['https://www.flashscore.nl/voetbal/europa/champions-league/schema/', 'https://www.worldfootball.net/all_matches/champions-league-2023-2024/'],
    'UEFA Europa League':['https://www.flashscore.nl/voetbal/europa/europa-league/schema/', 'https://www.worldfootball.net/all_matches/europa-league-2023-2024/'],
    'UEFA Europa Conference League':['https://www.flashscore.nl/voetbal/europa/europa-conference-league/schema/', 'https://www.worldfootball.net/all_matches/europa-conference-league-2023-2024/'],
}

In [20]:
testleagues_list = {
    #'English Premier League':['https://www.flashscore.nl/voetbal/engeland/premier-league/schema/', 'https://www.worldfootball.net/all_matches/eng-premier-league-2023-2024/'],
    #'EFL Championship':['https://www.flashscore.nl/voetbal/engeland/efl-cup/schema/', ''],
    #'EFL Trophy':['https://www.flashscore.nl/voetbal/engeland/efl-trophy/schema/', 'https://www.worldfootball.net/all_matches/eng-efl-trophy-2023-2024/'],
    'Ligue 1':['https://www.flashscore.nl/voetbal/frankrijk/ligue-1/schema/', 'https://www.worldfootball.net/all_matches/fra-ligue-1-2023-2024/'],
    #'La Liga':['https://www.flashscore.nl/voetbal/spanje/laliga/schema/','https://www.worldfootball.net/all_matches/esp-primera-division-2023-2024/'],
    #'La Liga2':['https://www.flashscore.nl/voetbal/spanje/laliga2/schema/', 'https://www.worldfootball.net/all_matches/esp-segunda-division-2023-2024/']
    #'Serie B':['https://www.flashscore.nl/voetbal/italie/serie-b/schema/', 'https://www.worldfootball.net/all_matches/ita-serie-b-2023-2024/'],
    #'Major League Soccer':['https://www.flashscore.nl/voetbal/usa/mls/schema/', 'https://www.worldfootball.net/all_matches/usa-major-league-soccer-2023/'],
}

In [21]:
today = date.today()
tomorrow = date.today() + timedelta(days=4)

In [22]:
print(today, tomorrow)

2023-09-12 2023-09-16


In [23]:
%%time
final_dataset = match_extraction(testleagues_list, tomorrow)

All daily matches of Ligue 1 have been extracted!
Wall time: 9min 11s


In [49]:
final_dataset

,Date/Time,HomeTeam,AwayTeam,match_urls,home_urls,away_urls,League,home_team_matches,away_team_matches,head2head_matches,home_team_matchespattern,away_team_matchespattern
1,2023-09-16 16:00:00,Stade Rennes,Lille OSC,https://www.flashscore.nl/wedstrijd/x2Fmw2II/#...,https://www.flashscore.nl/wedstrijd/x2Fmw2II/#...,https://www.flashscore.nl/wedstrijd/x2Fmw2II/#...,Ligue 1,{},{},{},{},{}
2,2023-09-16 20:00:00,RC Lens,FC Metz,https://www.flashscore.nl/wedstrijd/ruGqvr3C/#...,https://www.flashscore.nl/wedstrijd/ruGqvr3C/#...,https://www.flashscore.nl/wedstrijd/ruGqvr3C/#...,Ligue 1,{},{},{},{},{}


In [54]:
pd.DataFrame(json.loads(list(final_dataset['head2head_matches'])[1]))

,date,league,home_club,away_club,home_club_goal,away_club_goal
0,13.03.22,L1,FC Metz,RC Lens,0,0
1,24.10.21,L1,RC Lens,FC Metz,4,1
2,14.03.21,L1,RC Lens,FC Metz,2,2
3,19.12.20,L1,FC Metz,RC Lens,2,0
4,09.02.19,L2,RC Lens,FC Metz,0,0
5,01.09.18,L2,FC Metz,RC Lens,2,0
6,28.08.18,COU,RC Lens,FC Metz,1,2
7,08.01.17,CDF,RC Lens,FC Metz,2,0
8,13.05.16,L2,RC Lens,FC Metz,1,0
9,01.08.15,L2,FC Metz,RC Lens,0,0


In [346]:
variab = json.loads(list(final_dataset['home_team_matchespattern'])[0])
variab['7']

{'team': {'tournament': ['FRANKRIJK: Ligue 1 - Speelronde 16'],
  'date': ['01.12.2021 21:00'],
  'teams': ['Stade Rennes', 'Lille OSC'],
  'match_score': ['1-2'],
  'goal': ['85'],
  'penalty(missed)': [],
  'redcard': [],
  'red-yellowcard': ['82'],
  'substitution': ['46', '60', '60', '61', '71'],
  'var': [],
  'yellowcard': ['65', '81', '91']},
 'opponent': {'tournament': ['FRANKRIJK: Ligue 1 - Speelronde 16'],
  'date': ['01.12.2021 21:00'],
  'teams': ['Stade Rennes', 'Lille OSC'],
  'match_score': ['1-2'],
  'goal': ['31', '45'],
  'penalty(missed)': [],
  'redcard': [],
  'red-yellowcard': [],
  'substitution': ['71', '71', '80', '91'],
  'var': [],
  'yellowcard': ['89']}}

In [ ]:
#Write the script to first check if a team in a particular match has already been gotten before and simply extract from the database if extracted in the past
#In the analysis, also check team performance specifically for a given league.
    #Hence link entries to given leagues played.


#### Loading the Match Scores and Details into the Database

In [151]:
#Extracting the data from the dataframe to load into the database multiple rows at a time
lim = final_dataset.shape[0]

match_data = []
for i in range(lim):
    match_data.append(final_dataset.iloc[i,:])

In [153]:
#PostgreSQL database connection parameters
connection_params = {
    "host": "testtech.postgres.database.azure.com",
    "port": "5432",
    "database": "postgres",
    "user": "testtech",
    "password": "George9042"
}

#Connect to PostgreSQL
connection = psycopg2.connect(**connection_params)
cursor = connection.cursor()

#Uncomment and run only when creating new table

#Meant to run only the first time (Drops the table if it already exists)
table_name = "historic_match"
drop_query = f"DROP TABLE IF EXISTS {table_name}"
cursor.execute(drop_query)

#Create the table in the database
create_query = '''CREATE TABLE IF NOT EXISTS historic_match (
    date VARCHAR,
    hometeam VARCHAR,
    awayteam VARCHAR,
    match_urls VARCHAR,
    home_urls VARCHAR,
    away_urls VARCHAR,
    league VARCHAR,
    home_team_matches JSONB,
    away_team_matches JSONB,
    head2head_matches JSONB,
    home_team_matchespattern JSONB,
    away_team_matchespattern JSONB
);'''
cursor.execute(create_query)

#Insert all the data into the table multiple rows at a time
insert_query = "INSERT INTO historic_match (date, hometeam, awayteam, match_urls, home_urls, away_urls, league, home_team_matches, away_team_matches, head2head_matches, home_team_matchespattern, away_team_matchespattern) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"
cursor.executemany(insert_query, match_data)

In [154]:
#Extracts the data loaded into the database
select_query = "SELECT * FROM historic_match;"
cursor.execute(select_query)

# Fetch all rows
rows = cursor.fetchall()

In [155]:
#Commit and close connection
connection.commit()
cursor.close()
connection.close()

In [156]:
#Convrting the data extracted to a DataFrame for wuick analysis
df = pd.DataFrame(rows, columns=['date', 'hometeam', 'awayteam', 'match_urls', 'home_urls', 'away_urls', 'league', 'home_team_matches', 'away_team_matches', 'head2head_matches', 'home_team_matchespattern', 'away_team_matchespattern'])
df

,date,hometeam,awayteam,match_urls,home_urls,away_urls,league,home_team_matches,away_team_matches,head2head_matches,home_team_matchespattern,away_team_matchespattern
0,2023-09-02 15:00:00,Brentford,AFC Bournemouth,https://www.flashscore.nl/wedstrijd/hh2ZdWJ6/#...,https://www.flashscore.nl/wedstrijd/hh2ZdWJ6/#...,https://www.flashscore.nl/wedstrijd/hh2ZdWJ6/#...,English Premier League,"{'date': ['29.08.23', '26.08.23', '19.08.23', ...","{'date': ['29.08.23', '26.08.23', '19.08.23', ...","{'date': ['14.01.23', '01.10.22', '22.05.21', ...","{'1': {'var': [], 'date': ['26.08.2023 15:00']...","{'1': {'var': [], 'date': ['29.08.2023 19:30']..."
1,2023-09-02 15:00:00,Burnley,Tottenham Hotspur,https://www.flashscore.nl/wedstrijd/E7jreAlJ/#...,https://www.flashscore.nl/wedstrijd/E7jreAlJ/#...,https://www.flashscore.nl/wedstrijd/E7jreAlJ/#...,English Premier League,"{'date': ['30.08.23', '27.08.23', '11.08.23', ...","{'date': ['29.08.23', '26.08.23', '19.08.23', ...","{'date': ['15.05.22', '23.02.22', '27.10.21', ...","{'1': {'var': [], 'date': ['27.08.2023 14:00']...","{'1': {'var': [], 'date': ['29.08.2023 19:45']..."
2,2023-09-02 15:00:00,Chelsea,Nottingham Forest,https://www.flashscore.nl/wedstrijd/0d8k37tt/#...,https://www.flashscore.nl/wedstrijd/0d8k37tt/#...,https://www.flashscore.nl/wedstrijd/0d8k37tt/#...,English Premier League,"{'date': ['30.08.23', '25.08.23', '20.08.23', ...","{'date': ['30.08.23', '26.08.23', '18.08.23', ...","{'date': ['13.05.23', '01.01.23', '05.01.20', ...","{'1': {'var': [], 'date': ['30.08.2023 19:45']...","{'1': {'var': [], 'date': ['26.08.2023 15:00']..."
3,2023-09-02 15:00:00,Manchester City,Fulham,https://www.flashscore.nl/wedstrijd/Uy06aPu5/#...,https://www.flashscore.nl/wedstrijd/Uy06aPu5/#...,https://www.flashscore.nl/wedstrijd/Uy06aPu5/#...,English Premier League,"{'date': ['27.08.23', '19.08.23', '16.08.23', ...","{'date': ['29.08.23', '26.08.23', '19.08.23', ...","{'date': ['30.04.23', '05.11.22', '05.02.22', ...","{'1': {'var': [], 'date': ['19.08.2023 20:00']...","{'1': {}, '2': {'var': [], 'date': ['12.08.202..."
4,2023-09-02 17:30:00,Brighton,Newcastle United,https://www.flashscore.nl/wedstrijd/zHivdjZC/#...,https://www.flashscore.nl/wedstrijd/zHivdjZC/#...,https://www.flashscore.nl/wedstrijd/zHivdjZC/#...,English Premier League,"{'date': ['26.08.23', '19.08.23', '12.08.23', ...","{'date': ['27.08.23', '19.08.23', '12.08.23', ...","{'date': ['29.07.23', '18.05.23', '13.08.22', ...","{'1': {'var': [], 'date': ['26.08.2023 17:30']...","{'1': {'var': [], 'date': ['19.08.2023 20:00']..."
5,2023-09-16 12:30:00,Wolverhampton Wanderers,Liverpool,https://www.flashscore.nl/wedstrijd/tYvoBMvH/#...,https://www.flashscore.nl/wedstrijd/tYvoBMvH/#...,https://www.flashscore.nl/wedstrijd/tYvoBMvH/#...,English Premier League,"{'date': ['03.09.23', '29.08.23', '26.08.23', ...","{'date': ['03.09.23', '27.08.23', '19.08.23', ...","{'date': ['01.03.23', '04.02.23', '17.01.23', ...","{'1': {'var': [], 'date': ['29.08.2023 19:45']...","{'1': {}, '2': {'var': ['29'], 'date': ['13.08..."
6,2023-09-16 15:00:00,Aston Villa,Crystal Palace,https://www.flashscore.nl/wedstrijd/SUEBdNPN/#...,https://www.flashscore.nl/wedstrijd/SUEBdNPN/#...,https://www.flashscore.nl/wedstrijd/SUEBdNPN/#...,English Premier League,"{'date': ['03.09.23', '31.08.23', '27.08.23', ...","{'date': ['03.09.23', '29.08.23', '26.08.23', ...","{'date': ['04.03.23', '20.08.22', '15.05.22', ...",{},"{'1': {'var': [], 'date': ['29.08.2023 19:45']..."
7,2023-09-16 15:00:00,Fulham,Luton Town FC,https://www.flashscore.nl/wedstrijd/bD5si1mo/#...,https://www.flashscore.nl/wedstrijd/bD5si1mo/#...,https://www.flashscore.nl/wedstrijd/bD5si1mo/#...,English Premier League,"{'date': ['02.09.23', '29.08.23', '26.08.23', ...","{'date': ['01.09.23', '29.08.23', '25.08.23', ...","{'date': ['02.05.22', '11.12.21', '26.12.19', ...","{'1': {'var': [], 'date': ['29.08.2023 19:45']...","{'1': {'var': [], 'date': ['25.08.2023 20:00']..."
8,2023-09-16 15:00:00,Manchester U

In [162]:
list(df['home_team_matchespattern'])[5]['1']

{'var': [],
 'date': ['29.08.2023 19:45'],
 'goal': ['10', '25', '60', '66', '84'],
 'teams': ['Gaat doorWolverhampton Wanderers', 'Blackpool'],
 'redcard': [],
 'tournament': ['ENGELAND: EFL Cup - 32ste finale'],
 'yellowcard': ['40'],
 'match_score': ['5-0'],
 'substitution': ['55', '62', '81', '81'],
 'red-yellowcard': [],
 'penalty(missed)': []}

In [163]:
#Convert the column of the dataframe to a list first, before printing

print(json.loads(list(final_dataset['home_team_matchespattern'])[0])['1'])

{'tournament': ['ENGELAND: EFL Cup - 32ste finale'], 'date': ['29.08.2023 19:45'], 'teams': ['Gaat doorWolverhampton Wanderers', 'Blackpool'], 'match_score': ['5-0'], 'goal': ['10', '25', '60', '66', '84'], 'penalty(missed)': [], 'redcard': [], 'red-yellowcard': [], 'substitution': ['55', '62', '81', '81'], 'var': [], 'yellowcard': ['40']}


In [164]:
#Convert the column of the dataframe to a list first, before converting an entry to a datarame
x = list(final_dataset['home_team_matches'])[0]
h = json.loads(x)
h = pd.DataFrame(h)
h

,date,league,home_club,away_club,home_club_goal,away_club_goal
0,03.09.23,PL,Crystal Palace,Wolverhampton Wanderers,3,2
1,29.08.23,EFL,Wolverhampton Wanderers,Blackpool,5,0
2,26.08.23,PL,Everton,Wolverhampton Wanderers,0,1
3,19.08.23,PL,Wolverhampton Wanderers,Brighton,1,4
4,14.08.23,PL,Manchester United,Wolverhampton Wanderers,1,0
5,05.08.23,V,Wolverhampton Wanderers,Stade Rennes,3,1
6,02.08.23,V,Wolverhampton Wanderers,Luton Town FC,0,0
7,29.07.23,V,Celtic,Wolverhampton Wanderers,1,1
8,25.07.23,V,FC Porto,Wolverhampton Wanderers,0,1
9,09.07.23,V,Vitória SC,Wolverhampton Wanderers,1,2


---

### Referee Details Extraction

In [918]:
def refreehist_extraction(leagues_list, today_date):
    leagues_dataset = {} #Created the empyt dictionary that will be used to concatenate all table from all leagues

    for key in list(leagues_list.keys()): #Loops through all the leagues in our list of league url
        league_url = leagues_list[key][1]

        try:
            if league_url == '': 
                #Checks for empty league links and skips
                continue
            else:
                #Gets the contents from the leagues schedule page
                response = requests.get(league_url)
                time.sleep(2)
                soup = BeautifulSoup(response.content, "html.parser")

                #Extracts the table with the match schedules
                table = soup.find("table", class_="standard_tabelle")

                data = []
                match_links = []

                #Extracts the table rows
                table_rows = table.find_all("tr")
                for row in table_rows:
                    #Loops through al the rows and extracts the data in each columns
                    columns = row.find_all("td")
                    row_data = [column.get_text(strip=True) for column in columns]
                    data.append(row_data)

                    #Extracts the urls of each match
                    match_link = row.find("a", href=lambda href: href and "report" in href)
                    if match_link:
                        match_links.append(match_link["href"])
                    else:
                        match_links.append('')

                #Drops the empty entries
                zipped_lists = zip(data, match_links)
                zipped_lists = [pair for pair in zipped_lists if len(pair[0]) > 0]
                data, match_links = zip(*zipped_lists)

                columns = ['Date', 'Time', 'Home Team', 'Score', 'Away Team', 'Result', 'Links']
                df = pd.DataFrame(data, columns=columns)
                df.Links = match_links

                df = df[df['Links'] != ''] #Drops columns with empty url

                #Add the prefix to the column
                df['Links'] = 'https://www.worldfootball.net' + df['Links']
                df['Date'] = df['Date'].replace('', np.nan).ffill()
                df['Date'] = pd.to_datetime(df['Date'],  format='%d/%m/%Y')
                today = date.today()

                #Filter rows with today's date
                today_df = df[df['Date'].dt.date == today_date] #Account for when the dataset filter everything due to no matching date
                today_df = today_df.copy(deep=True)
                curr_league = [key for i in range(len(today_df['Links']))]
                today_df['League'] = curr_league

                #Extracts the link to the profile of the officiating referee from the match page using match url
                referee_urls = []
                for match_url in today_df.Links:
                    response = requests.get(match_url)
                    html_content = response.content
                    soup = BeautifulSoup(html_content, "html.parser")
                    referee_links = soup.find_all("a", href=lambda href: href and "referee_summary" in href)
                    ref_link = []
                    if len(referee_links) > 0:
                        for link in referee_links:
                            link_url = link.get("href")
                            ref_link.append(f'https://www.worldfootball.net{link_url}')
                            
                        ref_link = ref_link[0]
                        referee_urls.append(ref_link)
                    else:
                        referee_urls.append('')

                #Add the url of the profile of the officiating referee of each match to dataframe containing daily matches
                today_df['Referee_Links'] = referee_urls

                #Extracts the url of the most recent matches officiated by the officiating referee
                ref_matchhist_url = []
                for ref_url in today_df.Referee_Links:
                    if ref_url != '':
                        ref_matchhist = []

                        response = requests.get(ref_url)
                        html_content = response.content

                        soup = BeautifulSoup(html_content, "html.parser")

                        table = soup.find("table")
                        rows = table.find_all("tr")

                        for row in rows:
                            columns = row.find_all("td")
                            for column in columns:
                                (column.get_text())

                        referee_summary_links = soup.find_all("a", href=lambda href: href and "referee_summary" and "2023-2024" and "2022-2023" in href)

                        for link in referee_summary_links:
                            link_url = link.get("href")
                            ref_matchhist.append(f'https://www.worldfootball.net{link_url}')
                        ref_matchhist = [link for link in ref_matchhist if 'referee_summary' in link]
                        
                        ref_matchhist = {'1':ref_matchhist}
                        ref_matchhist = json.dumps(ref_matchhist)
                        ref_matchhist_url.append(ref_matchhist)
                    else:
                        ref_matchhist_url.append(json.dumps({}))

                #Add the urls of the most recent matches officiated by the referee to the dataframe of daily matches
                today_df['Referee_MatchHist_Links'] = ref_matchhist_url

                #Extracts the details from each match and stores in a dictionary
                ref_matchhist_detail = []
                for row in today_df.Referee_MatchHist_Links:
                    trans_row = json.loads(row)
                    if trans_row['1'] != []:
                        data_dict = {'Date':[], 'Home Team':[], 'Away Team':[], 'Score':[], 'Yellow Cards':[], 'Unkown Card':[], 'Red Cards':[]}
                        for ref_match_url in trans_row['1']:
                            response = requests.get(ref_match_url)
                            soup = BeautifulSoup(response.content, "html.parser")

                            table = soup.find("table", class_="standard_tabelle")

                            data = [] # List to store table data

                            table_rows = table.find_all("tr")
                            for row in table_rows:
                                columns = row.find_all("td")
                                row_data = [column.get_text(strip=True) for column in columns]
                                data.append(row_data)

                            data = data[1:]
                            for entry in data:
                                entry.pop(2)

                            for entry in data:
                                for i in range(len(entry)):
                                    keys = list(data_dict.keys())[i]
                                    data_dict[keys].append(entry[i])
                        data_dict = json.dumps(data_dict)
                        ref_matchhist_detail.append(data_dict)
                    else:
                        ref_matchhist_detail.append(json.dumps({}))

                #Extracted match details are added to the dataframe of daily matches
                today_df['Referee_MatchHist_Details'] = ref_matchhist_detail

            leagues_dataset[key] = today_df #Adds the dataframe of daily matches for a league to the dictionary of leagues
        except:
            continue

    #All the dataframe of the daily matches for all the leagues extracted are concatenated vertically
    list_of_keys = list(leagues_dataset.keys())
    if len(list_of_keys) > 0:
        for i in range(len(list_of_keys)):
            if i == 0:
                key = list_of_keys[i]
                final_dataset = leagues_dataset[key].copy(deep=True)
            else:
                key = list_of_keys[i]
                final_dataset = pd.concat([final_dataset, leagues_dataset[key]], axis=0)

        final_dataset.drop('Score', axis=1, inplace=True)
        return final_dataset
    else:
        print('No league scheduled matches were extracted!')

In [919]:
#Only extract matches with referees already announced.

referee_dataset = refreehist_extraction(testleagues_list, today)
referee_dataset

,Date,Time,Home Team,Away Team,Result,Links,League,Referee_Links,Referee_MatchHist_Links,Referee_MatchHist_Details
31,2023-09-02,12:30,Sheffield United,Everton FC,2:2 (2:1),https://www.worldfootball.net/report/premier-l...,English Premier League,https://www.worldfootball.net/referee_summary/...,"{""1"": [""https://www.worldfootball.net/referee_...","{""Date"": [""10/06/2022"", ""12/07/2022"", ""15/09/2..."
32,2023-09-02,15:00,Brentford FC,AFC Bournemouth,2:2 (1:1),https://www.worldfootball.net/report/premier-l...,English Premier League,https://www.worldfootball.net/referee_summary/...,"{""1"": [""https://www.worldfootball.net/referee_...","{""Date"": [""29/10/2022"", ""30/07/2022"", ""06/08/2..."
33,2023-09-02,15:00,Burnley FC,Tottenham Hotspur,2:5 (1:2),https://www.worldfootball.net/report/premier-l...,English Premier League,https://www.worldfootball.net/referee_summary/...,"{""1"": [""https://www.worldfootball.net/referee_...","{""Date"": [""21/09/2022"", ""24/09/2022"", ""27/10/2..."
34,2023-09-02,15:00,Chelsea FC,Nottingham Forest,0:1 (0:0),https://www.worldfootball.net/report/premier-l...,English Premier League,https://www.worldfootball.net/referee_summary/...,"{""1"": [""https://www.worldfootball.net/referee_...","{""Date"": [""04/03/2023"", ""01/04/2023"", ""30/07/2..."
35,2023-09-02,15:00,Manchester City,Fulham FC,5:1 (2:1),https://www.worldfootball.net/report/premier-l...,English Premier League,https://www.worldfootball.net/referee_summary/...,"{""1"": [""https://www.worldfootball.net/referee_...","{""Date"": [""02/06/2022"", ""13/09/2022"", ""11/10/2..."
36,2023-09-02,17:30,Brighton & Hove Albion,Newcastle United,0:0,https://www.worldfootball.net/report/premier-l...,English Premier League,https://www.worldfootball.net/referee_summary/...,"{""1"": [""https://www.worldfootball.net/referee_...","{""Date"": [""14/07/2022"", ""13/08/2022"", ""21/08/2..."


In [913]:
json.loads(list(referee_dataset['Referee_MatchHist_Links'])[1])['1']

['https://www.worldfootball.net/referee_summary/robert-madley/3/1/eng-premier-league-2022-2023/',
 'https://www.worldfootball.net/referee_summary/robert-madley/3/1/eng-championship-2022-2023/',
 'https://www.worldfootball.net/referee_summary/robert-madley/3/1/eng-league-cup-2022-2023/',
 'https://www.worldfootball.net/referee_summary/robert-madley/3/1/eng-league-one-2022-2023/',
 'https://www.worldfootball.net/referee_summary/robert-madley/3/1/eng-league-two-2022-2023/',
 'https://www.worldfootball.net/referee_summary/robert-madley/3/1/eng-playoffs-2022-2023-championship/',
 'https://www.worldfootball.net/referee_summary/robert-madley/3/1/eng-frauen-womens-super-league-2022-2023/']

In [914]:
pd.DataFrame(json.loads(list(referee_dataset['Referee_MatchHist_Details'])[1])).head(10)

,Date,Home Team,Away Team,Score,Yellow Cards,Unkown Card,Red Cards
0,29/10/2022,Brentford FC,Wolverhampton Wanderers,1:1,6,-,1
1,30/07/2022,Wigan Athletic,Preston North End,0:0,5,-,1
2,06/08/2022,Stoke City,Blackpool FC,2:0,4,-,-
3,13/08/2022,Millwall FC,Coventry City,3:2,6,1,-
4,16/08/2022,Burnley FC,Hull City,1:1,4,-,-
5,27/08/2022,Middlesbrough FC,Swansea City,2:1,7,1,-
6,08/10/2022,Blackpool FC,Watford FC,3:1,2,-,-
7,15/10/2022,Cardiff City,Coventry City,0:1,5,-,-
8,18/10/2022,West Bromwich Albion,Bristol City,0:2,4,-,-
9,23/10/2022,Watford FC,Luton Town,4:0,7,-,1


#### Loading Referee Details

In [921]:
#Extracting the data from the dataframe to load into the database multiple rows at a time
lim = referee_dataset.shape[0]

ref_data = []
for i in range(lim):
    ref_data.append(referee_dataset.iloc[i,:])

In [302]:
#PostgreSQL database connection parameters
connection_params = {
    "host": "testtech.postgres.database.azure.com",
    "port": "5432",
    "database": "postgres",
    "user": "testtech",
    "password": "George9042"
}

#Connect to PostgreSQL
connection = psycopg2.connect(**connection_params)
cursor = connection.cursor()

#Meant to run only the first time (Drops the table if it already exists)
table_name = "ref_historic_match"
drop_query = f"DROP TABLE IF EXISTS {table_name}"
cursor.execute(drop_query)

#Create the table in the database
create_query = '''CREATE TABLE IF NOT EXISTS ref_historic_match (
    date VARCHAR,
    time VARCHAR,
    hometeam VARCHAR,
    awayteam VARCHAR,
    result VARCHAR,
    matchlink VARCHAR,
    league VARCHAR,
    refereelink VARCHAR,
    referee_matchistlink JSONB,
    referee_matchhistdetails JSONB
);'''
cursor.execute(create_query)

#Insert all the data into the table multiple rows at a time
insert_query = "INSERT INTO ref_historic_match (date, time, hometeam, awayteam, result, matchlink, league, refereelink, referee_matchistlink, referee_matchhistdetails) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"
cursor.executemany(insert_query, ref_data)

In [303]:
#Extracts the data loaded into the database
select_query = "SELECT * FROM ref_historic_match;"
cursor.execute(select_query)

# Fetch all rows
rows = cursor.fetchall()

In [304]:
#Commit and close connection
connection.commit()
cursor.close()
connection.close()

In [305]:
#Convrting the data extracted to a DataFrame for wuick analysis
df_ref = pd.DataFrame(rows, columns=['date', 'time', 'hometeam', 'awayteam', 'result', 'matchlink', 'league', 'refereelink', 'referee_matchistlink', 'referee_matchhistdetails'])
df_ref

,date,time,hometeam,awayteam,result,matchlink,league,refereelink,referee_matchistlink,referee_matchhistdetails
0,2023-09-02 00:00:00,12:30,Sheffield United,Everton FC,2:2 (2:1),https://www.worldfootball.net/report/premier-l...,English Premier League,https://www.worldfootball.net/referee_summary/...,{'1': ['https://www.worldfootball.net/referee_...,"{'Date': ['10/06/2022', '12/07/2022', '15/09/2..."
1,2023-09-02 00:00:00,15:00,Brentford FC,AFC Bournemouth,2:2 (1:1),https://www.worldfootball.net/report/premier-l...,English Premier League,https://www.worldfootball.net/referee_summary/...,{'1': ['https://www.worldfootball.net/referee_...,"{'Date': ['29/10/2022', '30/07/2022', '06/08/2..."
2,2023-09-02 00:00:00,15:00,Burnley FC,Tottenham Hotspur,2:5 (1:2),https://www.worldfootball.net/report/premier-l...,English Premier League,https://www.worldfootball.net/referee_summary/...,{'1': ['https://www.worldfootball.net/referee_...,"{'Date': ['21/09/2022', '24/09/2022', '27/10/2..."
3,2023-09-02 00:00:00,15:00,Chelsea FC,Nottingham Forest,0:1 (0:0),https://www.worldfootball.net/report/premier-l...,English Premier League,https://www.worldfootball.net/referee_summary/...,{'1': ['https://www.worldfootball.net/referee_...,"{'Date': ['04/03/2023', '01/04/2023', '30/07/2..."
4,2023-09-02 00:00:00,15:00,Manchester City,Fulham FC,5:1 (2:1),https://www.worldfootball.net/report/premier-l...,English Premier League,https://www.worldfootball.net/referee_summary/...,{'1': ['https://www.worldfootball.net/referee_...,"{'Date': ['02/06/2022', '13/09/2022', '11/10/2..."
5,2023-09-02 00:00:00,17:30,Brighton & Hove Albion,Newcastle United,0:0,https://www.worldfootball.net/report/premier-l...,English Premier League,https://www.worldfootball.net/referee_summary/...,{'1': ['https://www.worldfootball.net/referee_...,"{'Date': ['14/07/2022', '13/08/2022', '21/08/2..."


---

### Historic Scores and Inner Match Details Analysis and Pattern Detection

In [167]:
match_df = df.copy(deep=True)

In [306]:
ref_df = df_ref.copy(deep=True)

In [439]:
match_df.tail(3)

,date,hometeam,awayteam,match_urls,home_urls,away_urls,league,home_team_matches,away_team_matches,head2head_matches,home_team_matchespattern,away_team_matchespattern
9,2023-09-16 15:00:00,Tottenham Hotspur,Sheffield United,https://www.flashscore.nl/wedstrijd/nZoxDrA4/#...,https://www.flashscore.nl/wedstrijd/nZoxDrA4/#...,https://www.flashscore.nl/wedstrijd/nZoxDrA4/#...,English Premier League,"{'date': ['02.09.23', '29.08.23', '26.08.23', ...","{'date': ['02.09.23', '30.08.23', '27.08.23', ...","{'date': ['01.03.23', '02.05.21', '17.01.21', ...","{'1': {'var': [], 'date': ['19.08.2023 17:30']...","{'1': {'var': [], 'date': ['18.08.2023 19:45']..."
10,2023-09-16 15:00:00,West Ham United,Manchester City,https://www.flashscore.nl/wedstrijd/UcysC2PA/#...,https://www.flashscore.nl/wedstrijd/UcysC2PA/#...,https://www.flashscore.nl/wedstrijd/UcysC2PA/#...,English Premier League,"{'date': ['01.09.23', '26.08.23', '20.08.23', ...","{'date': ['02.09.23', '27.08.23', '19.08.23', ...","{'date': ['03.05.23', '07.08.22', '15.05.22', ...","{'1': {'var': [], 'date': ['20.08.2023 16:30']...","{'1': {'var': [], 'date': ['27.08.2023 14:00']..."
11,2023-09-16 17:30:00,Newcastle United,Brentford,https://www.flashscore.nl/wedstrijd/v9rUE4uh/#...,https://www.flashscore.nl/wedstrijd/v9rUE4uh/#...,https://www.flashscore.nl/wedstrijd/v9rUE4uh/#...,English Premier League,"{'date': ['02.09.23', '27.08.23', '19.08.23', ...","{'date': ['02.09.23', '29.08.23', '26.08.23', ...","{'date': ['08.04.23', '08.10.22', '26.02.22', ...","{'1': {'var': [], 'date': ['27.08.2023 16:30']...","{'1': {'var': [], 'date': ['29.08.2023 19:45']..."


In [449]:
def indiv_teamrole_analysis(team_matches, team_name, role):
    '''This function filters the table by only matches played by the team under analysis where the said
    team played the role they will play in their upcoming match'''
    
    team_matches = json.loads(team_matches)
    team_df = pd.DataFrame(team_matches)
    team_df['home_club_goal'] = team_df['home_club_goal'].astype(int)
    team_df['away_club_goal'] = team_df['away_club_goal'].astype(int)
        
    #Creating the first part of the final string to describe the prediction depending on which set of historical matches (home, away of head-to-head)
    list_of_clubs = list(team_df['home_club']) + list(team_df['away_club'])
    num_of_club = len(set(list_of_clubs))
    if num_of_club > 2:
        end_string = 'Score prediction for {} team using {} historic match scores, for only matches where {} team played {} role: '.format(role, role, role, role)
    else:
        end_string = 'Score prediction for both teams using head-to-head historic match scores, for only matches where they played the same role as their upcoming game: '
    
    if role == 'home':
        if len(list(team_df[team_df['home_club']==team_name])) > 2:
            third_recentscore = [list(team_df[team_df['home_club']==team_name]['home_club_goal'])[2], list(team_df[team_df['away_club']!=team_name]['away_club_goal'])[2]]
        else:
            third_recentscore = []
    else:
        if len(list(team_df[team_df['away_club']==team_name])) > 2:
            third_recentscore = [list(team_df[team_df['home_club']!=team_name]['home_club_goal'])[2], list(team_df[team_df['away_club']==team_name]['away_club_goal'])[2]]
        else:
            third_recentscore = []
            
    def check(dataframe, column1, column2):
        varb = list((team_df[team_df[column1] == team_name])[column2])[:3] #Filter by a given team in a given role
        if len(varb) <= 1: #Checks if there's enough data to check for a pattern
            return ['-']
        else:
            #Checks for all the patterns for a particular role regardless of which team played
            if (varb[0] == varb[1]):
                return [varb[0]]
            elif ((max(varb[:2]) - min(varb[:2])) == 1):
                if 0 in varb[:2]:
                    return [max(varb[:2])+1]
                else:
                    return [max(varb[:2])+1, min(varb[:2])-1]
            elif ((max(varb[:2]) - min(varb[:2])) == 2):
                return [max(varb[:2])-1]
            else:
                return ['-']
    
    scores = []
    if role == 'home':
        scores.append(check(team_df, 'home_club', 'home_club_goal'))
        scores.append(check(team_df, 'home_club', 'away_club_goal'))
    else:
        scores.append(check(team_df, 'away_club', 'home_club_goal'))
        scores.append(check(team_df, 'away_club', 'away_club_goal'))
        
    combined_scores = []
    final_output = []
    if ['-'] not in scores:
        #Gets all the possible predictions from the observed patterns if any
        for score1 in scores[0]:
            for score2 in scores[1]:
                combined_scores.append([score1, score2])
                
        #Checks and removes the third most recent match outcome if it already exists in the list of possible predictions
        if third_recentscore in combined_scores:
            pass
        else:
            for score in combined_scores:
                final_string = end_string + '{} - {}'.format(score[0], score[1])
                final_output.append(final_string)            
    else:
        pass
    return final_output

In [450]:
def indiv_role_analysis(team_matches, role, skip=False):
    '''This function analyses the table without any filters and simple checks for pattern for a given
    role (home or away). The historic matches of the team to play the given role in the upcoming match is
    analysed for the given role'''
    
    team_matches = json.loads(team_matches)
    team_df = pd.DataFrame(team_matches)
    team_df['home_club_goal'] = team_df['home_club_goal'].astype(int)
    team_df['away_club_goal'] = team_df['away_club_goal'].astype(int)
    
    #Creating the first part of the final string to describe the prediction depending on which set of historical matches (home, away of head-to-head)
    list_of_clubs = list(team_df['home_club']) + list(team_df['away_club'])
    num_of_club = len(set(list_of_clubs))
    if num_of_club > 2:
        end_string = 'Score prediction for {} role using {} historic match scores, regardless of role played by {} team: '.format(role, role, role)
    else:
        end_string = 'Score prediction for both roles using head-to-head historic match scores, regardless of role played by both teams: '
    
    if skip == True:
        for i in range(team_df.shape[0]):
            if (i%2) == 0:
                team_df.drop(i, inplace=True)
                
        if num_of_club > 2:
            end_string = 'Score prediction for {} role using {} historic match scores after skipping rows, regardless of role played by {} team: '.format(role, role, role)
        else:
            end_string = 'Score prediction for both roles using head-to-head historic match scores after skipping rows, regardless of role played by both teams: '

    
    third_recentscore = [list(team_df['home_club_goal'])[2], list(team_df['away_club_goal'])[2]]
    
    def check(dataframe, column):
        varb = list(dataframe[column])[:3]
        if len(varb) <= 1: #Checks if there's enough data to check for a pattern
            return ['-']
        else:
            #Checks for all the patterns for a particular role regardless of which team played
            if (varb[0] == varb[1]):
                return [varb[0]]
            elif ((max(varb[:2]) - min(varb[:2])) == 1):
                if 0 in varb[:2]:
                    return [max(varb[:2])+1]
                else:
                    return [max(varb[:2])+1, min(varb[:2])-1]
            elif ((max(varb[:2]) - min(varb[:2])) == 2):
                return [max(varb[:2])-1]
            else:
                return ['-']
    
    scores = []    
    scores.append(check(team_df, 'home_club_goal'))
    scores.append(check(team_df, 'away_club_goal'))
    
    combined_scores = []
    final_output = []
    if ['-'] not in scores:
        #Gets all the possible predictions from the observed patterns if any
        for score1 in scores[0]:
            for score2 in scores[1]:
                combined_scores.append([score1, score2])
                
        #Checks and removes the third most recent match outcome if it already exists in the list of possible predictions
        if third_recentscore in combined_scores:
            pass
        else:
            for score in combined_scores:
                final_string = end_string + '{} - {}'.format(score[0], score[1])
                final_output.append(final_string)
    else:
        pass
    return final_output

In [451]:
def indiv_team_analysis(team_matches, team_name, role, skip=False):
    '''This function analyses a given team regardless of which role they played in their historic matches.
    It simply checks how they performed against any opponent and if there are any underlying patterns'''
    
    team_matches = json.loads(team_matches)
    team_df = pd.DataFrame(team_matches)
    team_df['home_club_goal'] = team_df['home_club_goal'].astype(int)
    team_df['away_club_goal'] = team_df['away_club_goal'].astype(int)
    
    #Creating the first part of the final string to describe the prediction depending on which set of historical matches (home, away of head-to-head)
    list_of_clubs = list(team_df['home_club']) + list(team_df['away_club'])
    num_of_club = len(set(list_of_clubs))
    if num_of_club > 2:
        end_string = 'Score prediction for {} team using {} historic match scores, regardless of role played by {} team: '.format(role, role, role)
    else:
        end_string = 'Score prediction for both teams using head-to-head historic match scores, regardless of role played by both teams: '
    
    if skip == True:
        for i in range(team_df.shape[0]):
            if (i%2) == 0:
                team_df.drop(i, inplace=True)
        
        if num_of_club > 2:
            end_string = 'Score prediction for {} team using {} historic match scores after skipping rows, regardless of role played by {} team: '.format(role, role, role)
        else:
            end_string = 'Score prediction for both teams using head-to-head historic match scores after skipping rows, regardless of role played by both teams: '
        
    
    rearranged = []
    #Rearrangement of the historic table to put the team under analysis in one side and all opponent on the other side.
    for i in range(team_df.shape[0]):
        if list(team_df.iloc[i,:])[2] == team_name:
            rearranged.append(list(team_df.iloc[i,:]))
        else:
            temp_list = list(team_df.iloc[i,:])
            temp_var = temp_list[2]
            temp_list[2] = temp_list[3]
            temp_list[3] = temp_var
            temp_var = temp_list[4]
            temp_list[4] = temp_list[5]
            temp_list[5] = temp_var
            rearranged.append(temp_list)
    new_team_df = pd.DataFrame(rearranged, columns=['date','league','team','opponent','team_score','opponent_score'])
    
    if role == 'home':
        third_recentscore = [list(new_team_df['team_score'])[2], list(new_team_df['opponent_score'])[2]]
    else:
        third_recentscore = [list(new_team_df['opponent_score'])[2], list(new_team_df['team_score'])[2]]
    
    def check(dataframe, column):
        varb = list(dataframe[column])[:3]
        if len(varb) <= 1: #Checks if there's enough data to check for a pattern
            return ['-']
        else:
            #Checks for all the patterns for a particular role regardless of which team played
            if (varb[0] == varb[1]):
                return [varb[0]]
            elif ((max(varb[:2]) - min(varb[:2])) == 1):
                if 0 in varb[:2]:
                    return [max(varb[:2])+1]
                else:
                    return [max(varb[:2])+1, min(varb[:2])-1]
            elif ((max(varb[:2]) - min(varb[:2])) == 2):
                return [max(varb[:2])-1]
            else:
                return ['-']
            
    scores = []    
    scores.append(check(new_team_df, 'team_score'))
    scores.append(check(new_team_df, 'opponent_score'))
    
    combined_scores = []
    final_output = []
    if ['-'] not in scores:
        if role == 'home':
            for score1 in scores[0]:
                for score2 in scores[1]:
                    combined_scores.append([score1, score2])
        else:
            for score1 in scores[1]:
                for score2 in scores[0]:
                    combined_scores.append([score1, score2])
                    
        #Checks and removes the third most recent match outcome if it already exists in the list of possible predictions
        if third_recentscore in combined_scores:
            pass
        else:
            for score in combined_scores:
                final_string = end_string + '{} - {}'.format(score[0], score[1])
                final_output.append(final_string)
    else:
        pass
    return final_output

In [452]:
def samescore_analysis(team_matches, role, skip=False):
    '''This function analyses the scores of the historic match regardless of team or role and regardless
    of orientation of the scores. Hence if simply gets the match scores and reorders them in descending order
    and check if there's a pattern (repetition)'''
    
    team_matches = json.loads(team_matches)
    team_df = pd.DataFrame(team_matches)
    team_df['home_club_goal'] = team_df['home_club_goal'].astype(int)
    team_df['away_club_goal'] = team_df['away_club_goal'].astype(int)
    
    if skip == True:
        for i in range(team_df.shape[0]):
            if (i%2) == 0:
                team_df.drop(i, inplace=True)
    
    #Check if there is a repeat in score, regardless of team playing a given role or the orientation (home before away)
    scores = []
    for i in range(3):
        temp_scorelist = [list(team_df['home_club_goal'])[i], list(team_df['away_club_goal'])[i]]
        scores.append([max(temp_scorelist), min(temp_scorelist)])
    
    if scores[0] == scores[1]:
        if scores[2] != scores[0]:
            print('Repeated score pattern ({} - {}) observed based on {} team history'.format(score[0][0], score[0][1], role))
        else:
            print('Repeated score pattern observed and cycle alread completed observed based on {} team history'.format(role))
    else:
        print('No pattern observed based on {} team history'.format(role))

In [453]:
def matchoutcome_analysis(team_matches, team_name, role, skip=False):
    '''This function analyses the historic match outcome type (win, loss, draw) for the team under analysis
    to see if there is an existing pattern.'''
    
    team_matches = json.loads(team_matches)
    team_df = pd.DataFrame(team_matches)
    team_df['home_club_goal'] = team_df['home_club_goal'].astype(int)
    team_df['away_club_goal'] = team_df['away_club_goal'].astype(int)
    
    
    if skip == True:
        for i in range(team_df.shape[0]):
            if (i%2) == 0:
                team_df.drop(i, inplace=True)
    
    rearranged = []
    #Rearrangement
    for i in range(team_df.shape[0]):
        if list(team_df.iloc[i,:])[2] == team_name:
            rearranged.append(list(team_df.iloc[i,:]))
        else:
            temp_list = list(team_df.iloc[i,:])
            temp_var = temp_list[2]
            temp_list[2] = temp_list[3]
            temp_list[3] = temp_var
            temp_var = temp_list[4]
            temp_list[4] = temp_list[5]
            temp_list[5] = temp_var
            rearranged.append(temp_list)
    new_team_df = pd.DataFrame(rearranged, columns=['date','league','team','opponent','team_score','opponent_score'])
    
    scores = []
    for i in range(3):
        temp_scorelist = [list(new_team_df['team_score'])[i], list(new_team_df['opponent_score'])[i]]
        scores.append(temp_scorelist)
        
    individ_scores = []
    for score in scores:
        for individ_score in score:
            individ_scores.append(individ_score)
        
    possible_outcomes = ['win', 'loss', 'draw']
    
    outcomes = []
    for score in scores:
        if score[0] == score[1]:
            outcomes.append('draw')
        elif score[0] > score[1]:
            outcomes.append('win')
        else:
            outcomes.append('loss')
    
    if outcomes[2] in [outcomes[0], outcomes[1]]:
        if outcomes[0] != outcomes[1]:
            prediction = [outcome for outcome in possible_outcomes if outcome not in outcomes]
            print('Prediction based on {} team history (idependent of role): {}'.format(role, prediction[0]))
        else:
            print('No pattern observed based on {} team history'.format(role))
    else:
        print('No pattern observed based on {} team history'.format(role))

In [454]:
def inner_detail_analyser(given_list):
    '''This function takes in the inner match details of the historical matchs of a given team and analyses
    it to extracts certain details to help with prediction.'''
    list_of_analysis = []
    for item in given_list:
        item_dict =json.loads(item)
        inner_detail = {'first_ten_minutes':[], 'last_ten_minutes':[], 'first_half':[],
                        'second_half':[], 'injury_time':[], 'avg_first_ten_minutes':[],
                        'avg_last_ten_minutes':[], 'avg_first_half':[], 'avg_second_half':[],
                        'avg_injury_time':[], 'match_firstgoal':[], 'match_lastgoal':[],
                        'underdog_effect':[]
                       }

        for key in list(item_dict.keys()):
            if item_dict[key] != {}:
                teamgoal_time = item_dict[key]['team']['goal']
                opponentgoal_time = item_dict[key]['opponent']['goal']
                count_firstten = 0
                count_lastten = 0
                count_firsthalf = 0
                count_secondhalf = 0
                count_injurytime = 0

                #Checks which of the goals scored at certain times during the match fit which category
                for minute in teamgoal_time:
                    if int(minute) < 10:
                        inner_detail['first_ten_minutes'].append(minute)
                        count_firstten += 1
                    if int(minute) < 49:
                        inner_detail['first_half'].append(minute)
                        count_firsthalf += 1
                    if int(minute) > 49:
                        inner_detail['second_half'].append(minute)
                        count_secondhalf += 1
                    if int(minute) > 80:
                        inner_detail['last_ten_minutes'].append(minute)
                        count_lastten += 1
                    if (int(minute) > 45) & (int(minute) < 50) | (int(minute) > 90):
                        inner_detail['injury_time'].append(minute)
                        count_injurytime += 1

                #Collates the number of these phenomena per match to later calculate the average
                inner_detail['avg_first_ten_minutes'].append(count_firstten)
                inner_detail['avg_last_ten_minutes'].append(count_lastten)
                inner_detail['avg_first_half'].append(count_firsthalf)
                inner_detail['avg_second_half'].append(count_secondhalf)
                inner_detail['avg_injury_time'].append(count_injurytime)

                team_goal = [int(minute) for minute in teamgoal_time]
                opponent_goal = [int(minute) for minute in opponentgoal_time]

                if (len(team_goal) > 0) & (len(opponent_goal)):
                    #Checks if the team scored the first goal
                    if min(team_goal) < min(opponent_goal):
                        inner_detail['match_firstgoal'].append('1')

                    #Checks if the team scored the last goal
                    if max(team_goal) > max(opponent_goal):
                        inner_detail['match_lastgoal'].append('1')

                    #Checks if the team conceded the first goal but still won the match
                    if (min(team_goal) > min(opponent_goal)) & (len(team_goal) > len(opponent_goal)):
                        inner_detail['underdog_effect'].append('1')  

        inner_detailanalysis = {}
        first_list = ['first_ten_minutes', 'last_ten_minutes', 'first_half', 'second_half', 'injury_time', 
                     'match_firstgoal', 'match_lastgoal', 'underdog_effect', ]
        second_list = ['avg_first_ten_minutes', 'avg_last_ten_minutes', 'avg_first_half', 'avg_second_half', 
                      'avg_injury_time', ]
        for key in inner_detail.keys():
            if key in first_list:
                inner_detailanalysis[key] = len(inner_detail[key])
            if key in second_list:
                inner_detailanalysis[key] = sum(inner_detail[key])/len(inner_detail[key])

        #print(inner_detail)
        list_of_analysis.append(inner_detailanalysis)
    list_of_analysis = pd.DataFrame(list_of_analysis)
    return list_of_analysis

In [455]:
def final_innerdet(innerdetail_df):
    '''This function compares the inner match details of both teams to play, and extracts the dominant team
    for each category.'''
    final_df = {}
    for column in list(innerdetail_df.columns):
        if (max(list(innerdetail_df[column])) == list(innerdetail_df[column])[0]) & (max(list(innerdetail_df[column])) == list(innerdetail_df[column])[1]):
            final_df[column] = ['both teams']
        elif (max(list(innerdetail_df[column])) == list(innerdetail_df[column])[0]):
            final_df[column] = ['home team']
        else:
            final_df[column] = ['away team']
    
    #final_df = pd.DataFrame(final_df)
    return final_df

In [478]:
def matchscore_total_analysis(dataset):
    '''This function takes in an entire row of the dataset pulled from the database and 
    extracts all the observed pattern from the historic match scores, as well as al the details
    from the inner match details of the historic matches for the home and awar team and their
    head-to-head meatches.'''
    diction = {'7':[1,'home','home_score_patterns'], '8':[2,'away','away_score_patterns'], '9':[1,'home','h2h_score_patterns']}
    columns_list = [7,8,9]
    dict_of_patterns = {'home_score_patterns':[], 'away_score_patterns':[], 'h2h_score_patterns':[],
                        'innerdetail_analysis':[]}
    for i in range(dataset.shape[0]):
        row = list(dataset.iloc[i,:])
        for number in columns_list:
            dict_of_pattern = {}
            list_of_pattern = []
            patterns = indiv_teamrole_analysis(row[number], row[diction[str(number)][0]], diction[str(number)][1])
            list_of_pattern = list_of_pattern + [pattern for pattern in patterns]
            patterns = indiv_role_analysis(row[number], diction[str(number)][1])
            list_of_pattern = list_of_pattern + [pattern for pattern in patterns]
            patterns = indiv_role_analysis(row[number], diction[str(number)][1], skip=True)
            list_of_pattern = list_of_pattern + [pattern for pattern in patterns]
            patterns = indiv_team_analysis(row[number], row[diction[str(number)][0]], diction[str(number)][1])
            list_of_pattern = list_of_pattern + [pattern for pattern in patterns]
            patterns = indiv_team_analysis(row[number], row[diction[str(number)][0]], diction[str(number)][1], skip=True)
            list_of_pattern = list_of_pattern + [pattern for pattern in patterns]

            for i in range(len(list_of_pattern)):
                dict_of_pattern[str(i)] = list_of_pattern[i]

            dict_of_patterns[diction[str(number)][2]].append(dict_of_pattern)


        innerdetail_df = inner_detail_analyser(row[-2:])
        innerdetail = final_innerdet(innerdetail_df)
        dict_of_patterns['innerdetail_analysis'].append(innerdetail)
    return dict_of_patterns

---

In [471]:
final_dataset

,Date/Time,HomeTeam,AwayTeam,match_urls,home_urls,away_urls,League,home_team_matches,away_team_matches,head2head_matches,home_team_matchespattern,away_team_matchespattern
1,2023-09-16 16:00:00,Stade Rennes,Lille OSC,https://www.flashscore.nl/wedstrijd/x2Fmw2II/#...,https://www.flashscore.nl/wedstrijd/x2Fmw2II/#...,https://www.flashscore.nl/wedstrijd/x2Fmw2II/#...,Ligue 1,"{""date"": [""02.09.23"", ""27.08.23"", ""20.08.23"", ...","{""date"": [""03.09.23"", ""31.08.23"", ""27.08.23"", ...","{""date"": [""04.02.23"", ""06.11.22"", ""21.05.22"", ...","{""1"": {""team"": {""tournament"": [""FRANKRIJK: Lig...","{""1"": {""team"": {""tournament"": [""EUROPA: Europa..."
2,2023-09-16 20:00:00,RC Lens,FC Metz,https://www.flashscore.nl/wedstrijd/ruGqvr3C/#...,https://www.flashscore.nl/wedstrijd/ruGqvr3C/#...,https://www.flashscore.nl/wedstrijd/ruGqvr3C/#...,Ligue 1,"{""date"": [""02.09.23"", ""26.08.23"", ""20.08.23"", ...","{""date"": [""08.09.23"", ""03.09.23"", ""27.08.23"", ...","{""date"": [""13.03.22"", ""24.10.21"", ""14.03.21"", ...","{""1"": {""team"": {""tournament"": [""FRANKRIJK: Lig...","{""1"": {""team"": {""tournament"": [""FRANKRIJK: Lig..."


In [472]:
additional_columns = matchscore_total_analysis(final_dataset)

modified_dataset = final_dataset.copy(deep=True)
for key in additional_columns.keys():
    modified_dataset[key] = additional_columns[key]

In [473]:
modified_dataset

,Date/Time,HomeTeam,AwayTeam,match_urls,home_urls,away_urls,League,home_team_matches,away_team_matches,head2head_matches,home_team_matchespattern,away_team_matchespattern,home_score_patterns,away_score_patterns,h2h_score_patterns,innerdetail_analysis
1,2023-09-16 16:00:00,Stade Rennes,Lille OSC,https://www.flashscore.nl/wedstrijd/x2Fmw2II/#...,https://www.flashscore.nl/wedstrijd/x2Fmw2II/#...,https://www.flashscore.nl/wedstrijd/x2Fmw2II/#...,Ligue 1,"{""date"": [""02.09.23"", ""27.08.23"", ""20.08.23"", ...","{""date"": [""03.09.23"", ""31.08.23"", ""27.08.23"", ...","{""date"": [""04.02.23"", ""06.11.22"", ""21.05.22"", ...","{""1"": {""team"": {""tournament"": [""FRANKRIJK: Lig...","{""1"": {""team"": {""tournament"": [""EUROPA: Europa...",{},{'0': 'Score prediction for away role using aw...,{'0': 'Score prediction for both teams using h...,"{'first_ten_minutes': ['home team'], 'last_ten..."
2,2023-09-16 20:00:00,RC Lens,FC Metz,https://www.flashscore.nl/wedstrijd/ruGqvr3C/#...,https://www.flashscore.nl/wedstrijd/ruGqvr3C/#...,https://www.flashscore.nl/wedstrijd/ruGqvr3C/#...,Ligue 1,"{""date"": [""02.09.23"", ""26.08.23"", ""20.08.23"", ...","{""date"": [""08.09.23"", ""03.09.23"", ""27.08.23"", ...","{""date"": [""13.03.22"", ""24.10.21"", ""14.03.21"", ...","{""1"": {""team"": {""tournament"": [""FRANKRIJK: Lig...","{""1"": {""team"": {""tournament"": [""FRANKRIJK: Lig...",{'0': 'Score prediction for home team using ho...,{'0': 'Score prediction for away role using aw...,{'0': 'Score prediction for both teams using h...,"{'first_ten_minutes': ['both teams'], 'last_te..."


In [474]:
list(modified_dataset['home_score_patterns'])[1]

{'0': 'Score prediction for home team using home historic match scores, for only matches where home team played home role: 2 - 2',
 '1': 'Score prediction for home role using home historic match scores, regardless of role played by home team: 3 - 2',
 '2': 'Score prediction for home role using home historic match scores after skipping rows, regardless of role played by home team: 3 - 3',
 '3': 'Score prediction for home role using home historic match scores after skipping rows, regardless of role played by home team: 3 - 0',
 '4': 'Score prediction for home team using home historic match scores, regardless of role played by home team: 2 - 3',
 '5': 'Score prediction for home team using home historic match scores after skipping rows, regardless of role played by home team: 3 - 3',
 '6': 'Score prediction for home team using home historic match scores after skipping rows, regardless of role played by home team: 0 - 3'}

In [475]:
list(modified_dataset['away_score_patterns'])[1]

{'0': 'Score prediction for away role using away historic match scores, regardless of role played by away team: 3 - 3',
 '1': 'Score prediction for away role using away historic match scores, regardless of role played by away team: 3 - 0',
 '2': 'Score prediction for away role using away historic match scores, regardless of role played by away team: 0 - 3',
 '3': 'Score prediction for away role using away historic match scores, regardless of role played by away team: 0 - 0',
 '4': 'Score prediction for away role using away historic match scores after skipping rows, regardless of role played by away team: 2 - 2',
 '5': 'Score prediction for away team using away historic match scores, regardless of role played by away team: 3 - 3',
 '6': 'Score prediction for away team using away historic match scores, regardless of role played by away team: 3 - 0',
 '7': 'Score prediction for away team using away historic match scores, regardless of role played by away team: 0 - 3',
 '8': 'Score predict

In [476]:
list(modified_dataset['h2h_score_patterns'])[1]

{'0': 'Score prediction for both teams using head-to-head historic match scores, for only matches where they played the same role as their upcoming game: 3 - 3',
 '1': 'Score prediction for both teams using head-to-head historic match scores, for only matches where they played the same role as their upcoming game: 3 - 0',
 '2': 'Score prediction for both roles using head-to-head historic match scores after skipping rows, regardless of role played by both teams: 3 - 2'}

In [477]:
list(modified_dataset['innerdetail_analysis'])[1]

{'first_ten_minutes': ['both teams'],
 'last_ten_minutes': ['home team'],
 'first_half': ['home team'],
 'second_half': ['home team'],
 'injury_time': ['away team'],
 'avg_first_ten_minutes': ['home team'],
 'avg_last_ten_minutes': ['home team'],
 'avg_first_half': ['home team'],
 'avg_second_half': ['home team'],
 'avg_injury_time': ['away team'],
 'match_firstgoal': ['home team'],
 'match_lastgoal': ['home team'],
 'underdog_effect': ['both teams']}

In [266]:
samescore_analysis(row[7], 'home')
samescore_analysis(row[8], 'away')                 

No pattern observed based on home team history
No pattern observed based on away team history


In [267]:
samescore_analysis(row[9], 'home')
samescore_analysis(row[9], 'away') 

No pattern observed based on home team history
No pattern observed based on away team history


In [299]:
matchoutcome_analysis(row[7], row[1], 'home')
matchoutcome_analysis(row[8], row[2], 'away')

Prediction based on home team history (idependent of role): draw
No pattern observed based on away team history


In [300]:
matchoutcome_analysis(row[9], row[1], 'home')
matchoutcome_analysis(row[9], row[2], 'away')

Prediction based on home team history (idependent of role): draw
Prediction based on away team history (idependent of role): draw


In [ ]:
#Find out from client if samescore and matcoutcome function should be combined
#Since the two different histories (teams and referee) will be extracted separately, they should

---

In [512]:
pd.DataFrame(json.loads(list(modified_dataset['home_team_matches'])[1])).head(3)

,date,league,home_club,away_club,home_club_goal,away_club_goal
0,02.09.23,L1,AS Monaco,RC Lens,3,0
1,26.08.23,L1,PSG,RC Lens,3,1
2,20.08.23,L1,RC Lens,Stade Rennes,1,1


In [513]:
pd.DataFrame(json.loads(list(modified_dataset['away_team_matches'])[1])).head(3)

,date,league,home_club,away_club,home_club_goal,away_club_goal
0,08.09.23,V,FC Metz,FC Basel,1,1
1,03.09.23,L1,FC Metz,Stade de Reims,2,2
2,27.08.23,L1,Clermont,FC Metz,0,1


In [514]:
pd.DataFrame(json.loads(list(modified_dataset['head2head_matches'])[1])).head(3)

,date,league,home_club,away_club,home_club_goal,away_club_goal
0,13.03.22,L1,FC Metz,RC Lens,0,0
1,24.10.21,L1,RC Lens,FC Metz,4,1
2,14.03.21,L1,RC Lens,FC Metz,2,2


---

### Referee Histroy Analysis

In [499]:
ref_df.head(3)

,date,time,hometeam,awayteam,result,matchlink,league,refereelink,referee_matchistlink,referee_matchhistdetails
0,2023-09-02 00:00:00,12:30,Sheffield United,Everton FC,2:2 (2:1),https://www.worldfootball.net/report/premier-l...,English Premier League,https://www.worldfootball.net/referee_summary/...,{'1': ['https://www.worldfootball.net/referee_...,"{'Date': ['10/06/2022', '12/07/2022', '15/09/2..."
1,2023-09-02 00:00:00,15:00,Brentford FC,AFC Bournemouth,2:2 (1:1),https://www.worldfootball.net/report/premier-l...,English Premier League,https://www.worldfootball.net/referee_summary/...,{'1': ['https://www.worldfootball.net/referee_...,"{'Date': ['29/10/2022', '30/07/2022', '06/08/2..."
2,2023-09-02 00:00:00,15:00,Burnley FC,Tottenham Hotspur,2:5 (1:2),https://www.worldfootball.net/report/premier-l...,English Premier League,https://www.worldfootball.net/referee_summary/...,{'1': ['https://www.worldfootball.net/referee_...,"{'Date': ['21/09/2022', '24/09/2022', '27/10/2..."


In [493]:
def ref_hist_analysis(team_matches, skip=False):
    '''This function analyses the table without any filters and simple checks for pattern for a given
    role (home or away). The historic matches of the team to play the given role in the upcoming match is
    analysed for the given role'''
    
    #team_matches = json.loads(team_matches)
    team_df = pd.DataFrame(team_matches)
    team_df['home_club_goal'] = team_df['home_club_goal'].astype(int)
    team_df['away_club_goal'] = team_df['away_club_goal'].astype(int)
    
    end_string = 'Score prediction based on ref history: '
    
    if skip == True:
        for i in range(team_df.shape[0]):
            if (i%2) == 0:
                team_df.drop(i, inplace=True)
        end_string = 'Score prediction based on ref history after skipping rows: '
    
    third_recentscore = [list(team_df['home_club_goal'])[2], list(team_df['away_club_goal'])[2]]
    
    def check(dataframe, column):
        varb = list(dataframe[column])[:3]
        if len(varb) <= 1: #Checks if there's enough data to check for a pattern
            return ['-']
        else:
            #Checks for all the patterns for a particular role regardless of which team played
            if (varb[0] == varb[1]):
                return [varb[0]]
            elif ((max(varb[:2]) - min(varb[:2])) == 1):
                if 0 in varb[:2]:
                    return [max(varb[:2])+1]
                else:
                    return [max(varb[:2])+1, min(varb[:2])-1]
            elif ((max(varb[:2]) - min(varb[:2])) == 2):
                return [max(varb[:2])-1]
            else:
                return ['-']
    
    scores = []    
    scores.append(check(team_df, 'home_club_goal'))
    scores.append(check(team_df, 'away_club_goal'))
    
    combined_scores = []
    final_output = []
    if ['-'] not in scores:
        #Gets all the possible predictions from the observed patterns if any
        for score1 in scores[0]:
            for score2 in scores[1]:
                combined_scores.append([score1, score2])
                
        #Checks and removes the third most recent match outcome if it already exists in the list of possible predictions
        if third_recentscore in combined_scores:
            pass
        else:
            for score in combined_scores:
                final_string = end_string + '{} - {}'.format(score[0], score[1])
                final_output.append(final_string)
    else:
        pass
    return final_output

In [496]:
def ref_total_analysis(dataset):
    '''This function takes in an entire row of the dataset pulled from the database and 
    extracts all the observed patterns.'''
    dict_of_patterns = {'ref_patterns':[]}
    
    for i in range(dataset.shape[0]):
        ref_row = list(dataset.iloc[i,:])
        
        ref_hist = pd.DataFrame(ref_row[9])
        ref_hist['Date'] = pd.to_datetime(ref_hist['Date'], format='%d/%m/%Y')
        ref_hist[['home_club_goal', 'away_club_goal']] = ref_hist['Score'].str.split(':', 1, expand=True)
        ref_hist = ref_hist[~ref_hist['away_club_goal'].str.contains('pso')]
        ref_hist = ref_hist[~ref_hist['home_club_goal'].str.contains('pso')]
        ref_hist = ref_hist[~ref_hist['away_club_goal'].str.contains('aet')]
        ref_hist = ref_hist[~ref_hist['home_club_goal'].str.contains('aet')]
        ref_hist.sort_values(by='Date', ascending=False, inplace=True)
        ref_hist.reset_index(inplace=True)
        
        dict_of_pattern = {}
        list_of_pattern = []
        patterns = ref_hist_analysis(ref_hist)
        list_of_pattern = list_of_pattern + [pattern for pattern in patterns]
        patterns = ref_hist_analysis(ref_hist, skip=True)
        list_of_pattern = list_of_pattern + [pattern for pattern in patterns]
        
        for i in range(len(list_of_pattern)):
            dict_of_pattern[str(i)] = list_of_pattern[i]

        dict_of_patterns['ref_patterns'].append(dict_of_pattern)
    return dict_of_patterns

In [497]:
ref_additional_columns = ref_total_analysis(ref_df)

ref_modified_dataset = ref_df.copy(deep=True)
for key in ref_additional_columns.keys():
    ref_modified_dataset[key] = ref_additional_columns[key]

In [504]:
ref_modified_dataset.head(3)

,date,time,hometeam,awayteam,result,matchlink,league,refereelink,referee_matchistlink,referee_matchhistdetails,ref_patterns
0,2023-09-02 00:00:00,12:30,Sheffield United,Everton FC,2:2 (2:1),https://www.worldfootball.net/report/premier-l...,English Premier League,https://www.worldfootball.net/referee_summary/...,{'1': ['https://www.worldfootball.net/referee_...,"{'Date': ['10/06/2022', '12/07/2022', '15/09/2...",{'0': 'Score prediction based on ref history: ...
1,2023-09-02 00:00:00,15:00,Brentford FC,AFC Bournemouth,2:2 (1:1),https://www.worldfootball.net/report/premier-l...,English Premier League,https://www.worldfootball.net/referee_summary/...,{'1': ['https://www.worldfootball.net/referee_...,"{'Date': ['29/10/2022', '30/07/2022', '06/08/2...",{'0': 'Score prediction based on ref history a...
2,2023-09-02 00:00:00,15:00,Burnley FC,Tottenham Hotspur,2:5 (1:2),https://www.worldfootball.net/report/premier-l...,English Premier League,https://www.worldfootball.net/referee_summary/...,{'1': ['https://www.worldfootball.net/referee_...,"{'Date': ['21/09/2022', '24/09/2022', '27/10/2...",{'0': 'Score prediction based on ref history a...


In [503]:
list(ref_modified_dataset['ref_patterns'])[0]

{'0': 'Score prediction based on ref history: 1 - 2',
 '1': 'Score prediction based on ref history after skipping rows: 2 - 1'}

In [333]:
#Adapt the match outcome and score analyser to the referee history after confirming from the client whether to merge

---

### Linking Historic match dataset to Referee dataset

In [508]:
match_df.head(5)

,date,hometeam,awayteam,match_urls,home_urls,away_urls,league,home_team_matches,away_team_matches,head2head_matches,home_team_matchespattern,away_team_matchespattern
0,2023-09-02 15:00:00,Brentford,AFC Bournemouth,https://www.flashscore.nl/wedstrijd/hh2ZdWJ6/#...,https://www.flashscore.nl/wedstrijd/hh2ZdWJ6/#...,https://www.flashscore.nl/wedstrijd/hh2ZdWJ6/#...,English Premier League,"{'date': ['29.08.23', '26.08.23', '19.08.23', ...","{'date': ['29.08.23', '26.08.23', '19.08.23', ...","{'date': ['14.01.23', '01.10.22', '22.05.21', ...","{'1': {'var': [], 'date': ['26.08.2023 15:00']...","{'1': {'var': [], 'date': ['29.08.2023 19:30']..."
1,2023-09-02 15:00:00,Burnley,Tottenham Hotspur,https://www.flashscore.nl/wedstrijd/E7jreAlJ/#...,https://www.flashscore.nl/wedstrijd/E7jreAlJ/#...,https://www.flashscore.nl/wedstrijd/E7jreAlJ/#...,English Premier League,"{'date': ['30.08.23', '27.08.23', '11.08.23', ...","{'date': ['29.08.23', '26.08.23', '19.08.23', ...","{'date': ['15.05.22', '23.02.22', '27.10.21', ...","{'1': {'var': [], 'date': ['27.08.2023 14:00']...","{'1': {'var': [], 'date': ['29.08.2023 19:45']..."
2,2023-09-02 15:00:00,Chelsea,Nottingham Forest,https://www.flashscore.nl/wedstrijd/0d8k37tt/#...,https://www.flashscore.nl/wedstrijd/0d8k37tt/#...,https://www.flashscore.nl/wedstrijd/0d8k37tt/#...,English Premier League,"{'date': ['30.08.23', '25.08.23', '20.08.23', ...","{'date': ['30.08.23', '26.08.23', '18.08.23', ...","{'date': ['13.05.23', '01.01.23', '05.01.20', ...","{'1': {'var': [], 'date': ['30.08.2023 19:45']...","{'1': {'var': [], 'date': ['26.08.2023 15:00']..."
3,2023-09-02 15:00:00,Manchester City,Fulham,https://www.flashscore.nl/wedstrijd/Uy06aPu5/#...,https://www.flashscore.nl/wedstrijd/Uy06aPu5/#...,https://www.flashscore.nl/wedstrijd/Uy06aPu5/#...,English Premier League,"{'date': ['27.08.23', '19.08.23', '16.08.23', ...","{'date': ['29.08.23', '26.08.23', '19.08.23', ...","{'date': ['30.04.23', '05.11.22', '05.02.22', ...","{'1': {'var': [], 'date': ['19.08.2023 20:00']...","{'1': {}, '2': {'var': [], 'date': ['12.08.202..."
4,2023-09-02 17:30:00,Brighton,Newcastle United,https://www.flashscore.nl/wedstrijd/zHivdjZC/#...,https://www.flashscore.nl/wedstrijd/zHivdjZC/#...,https://www.flashscore.nl/wedstrijd/zHivdjZC/#...,English Premier League,"{'date': ['26.08.23', '19.08.23', '12.08.23', ...","{'date': ['27.08.23', '19.08.23', '12.08.23', ...","{'date': ['29.07.23', '18.05.23', '13.08.22', ...","{'1': {'var': [], 'date': ['26.08.2023 17:30']...","{'1': {'var': [], 'date': ['19.08.2023 20:00']..."


In [509]:
ref_df.tail(5)

,date,time,hometeam,awayteam,result,matchlink,league,refereelink,referee_matchistlink,referee_matchhistdetails
1,2023-09-02 00:00:00,15:00,Brentford FC,AFC Bournemouth,2:2 (1:1),https://www.worldfootball.net/report/premier-l...,English Premier League,https://www.worldfootball.net/referee_summary/...,{'1': ['https://www.worldfootball.net/referee_...,"{'Date': ['29/10/2022', '30/07/2022', '06/08/2..."
2,2023-09-02 00:00:00,15:00,Burnley FC,Tottenham Hotspur,2:5 (1:2),https://www.worldfootball.net/report/premier-l...,English Premier League,https://www.worldfootball.net/referee_summary/...,{'1': ['https://www.worldfootball.net/referee_...,"{'Date': ['21/09/2022', '24/09/2022', '27/10/2..."
3,2023-09-02 00:00:00,15:00,Chelsea FC,Nottingham Forest,0:1 (0:0),https://www.worldfootball.net/report/premier-l...,English Premier League,https://www.worldfootball.net/referee_summary/...,{'1': ['https://www.worldfootball.net/referee_...,"{'Date': ['04/03/2023', '01/04/2023', '30/07/2..."
4,2023-09-02 00:00:00,15:00,Manchester City,Fulham FC,5:1 (2:1),https://www.worldfootball.net/report/premier-l...,English Premier League,https://www.worldfootball.net/referee_summary/...,{'1': ['https://www.worldfootball.net/referee_...,"{'Date': ['02/06/2022', '13/09/2022', '11/10/2..."
5,2023-09-02 00:00:00,17:30,Brighton & Hove Albion,Newcastle United,0:0,https://www.worldfootball.net/report/premier-l...,English Premier League,https://www.worldfootball.net/referee_summary/...,{'1': ['https://www.worldfootball.net/referee_...,"{'Date': ['14/07/2022', '13/08/2022', '21/08/2..."


In [511]:
#Create a list to add blanks where there no match for a given team set up, and add referee analysis where there is a match
#After the loop, attach the referee analysis and referee name to the team analysis dataset.
for i in range(match_df.shape[0]):
    for j in range(ref_df.shape[0]):
        if (list(match_df.iloc[i,:])[1] in list(ref_df.iloc[j,:])[2]) & (list(match_df.iloc[i,:])[2] in list(ref_df.iloc[j,:])[3]):
            print('match_df: ', list(match_df.iloc[i,:])[1], list(match_df.iloc[i,:])[2])
            print('ref_df:   ', list(ref_df.iloc[j,:])[2], list(ref_df.iloc[j,:])[3])

match_df:  Brentford AFC Bournemouth
ref_df:    Brentford FC AFC Bournemouth
match_df:  Burnley Tottenham Hotspur
ref_df:    Burnley FC Tottenham Hotspur
match_df:  Chelsea Nottingham Forest
ref_df:    Chelsea FC Nottingham Forest
match_df:  Manchester City Fulham
ref_df:    Manchester City Fulham FC
match_df:  Brighton Newcastle United
ref_df:    Brighton & Hove Albion Newcastle United
